## 使用 @tool 裝飾器


In [67]:
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool


@tool
def search(query: str) -> str:
    """在線搜尋內容。"""
    return "LangChain"


#
print("1. search.name：", search.name)
#
print("2. search.description：", search.description)
#
print("3. search.args：", search.args)


@tool
def multiply(a: int, b: int) -> int:
    """將兩個數字相乘。"""
    return a * b


print("\n")
print("1. multiply.name：", multiply.name)
print("2. multiply.description：", multiply.description)
print("3. multiply.args：", multiply.args)

1. search.name： search
2. search.description： 在線搜尋內容。
3. search.args： {'query': {'title': 'Query', 'type': 'string'}}


1. multiply.name： multiply
2. multiply.description： 將兩個數字相乘。
3. multiply.args： {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


## 自訂工具名稱和 JSON 參數


In [68]:
class SearchInput(BaseModel):
    query: str = Field(description="應該是一個搜尋查詢")


@tool("search-tool", args_schema=SearchInput, return_direct=True)
def search(query: str) -> str:
    """在線搜尋內容。"""
    return "LangChain"


print('search.name：', search.name)
print('search.description：', search.description)
print('search.args：', search.args)
print('search.return_direct：', search.return_direct)

search.name： search-tool
search.description： 在線搜尋內容。
search.args： {'query': {'title': 'Query', 'description': '應該是一個搜尋查詢', 'type': 'string'}}
search.return_direct： True


## 繼承 BaseTool 類


In [69]:
from typing import Optional, Type

from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)


class SearchInput(BaseModel):
    query: str = Field(description="應該是一個搜尋查詢")


class CalculatorInput(BaseModel):
    a: int = Field(description="第一個數字")
    b: int = Field(description="第二個數字")


class CustomSearchTool(BaseTool):
    name = "custom_search"
    description = "當你需要回答關於當前事件的問題時很有用"
    args_schema: Type[BaseModel] = SearchInput

    def _run(
        self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """使用這個工具。"""
        return "LangChain"

    async def _arun(
        self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None
    ) -> str:
        """異步使用這個工具。"""
        raise NotImplementedError("custom_search 不支持異步")


class CustomCalculatorTool(BaseTool):
    name = "Calculator"
    description = "當你需要回答數學問題時很有用"
    args_schema: Type[BaseModel] = CalculatorInput
    return_direct: bool = True

    def _run(
        self, a: int, b: int, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """使用這個工具。"""
        return a * b

    async def _arun(
        self,
        a: int,
        b: int,
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> str:
        """異步使用這個工具。"""
        raise NotImplementedError("Calculator 不支持異步")


search = CustomSearchTool()
print("search.name", search.name)
print("search.description", search.description)
print("search.args", search.args)
print("\n")

multiply = CustomCalculatorTool()
print("multiply.name", multiply.name)
print("multiply.description", multiply.description)
print("multiply.args", multiply.args)
print("multiply.return_direct", multiply.return_direct)

search.name custom_search
search.description 當你需要回答關於當前事件的問題時很有用
search.args {'query': {'title': 'Query', 'description': '應該是一個搜尋查詢', 'type': 'string'}}


multiply.name Calculator
multiply.description 當你需要回答數學問題時很有用
multiply.args {'a': {'title': 'A', 'description': '第一個數字', 'type': 'integer'}, 'b': {'title': 'B', 'description': '第二個數字', 'type': 'integer'}}
multiply.return_direct True


## 使用 StructuredTool 類


In [70]:
def search_function(query: str):
    return "LangChain"


search = StructuredTool.from_function(
    func=search_function,
    name="Search",
    description="當你需要回答關於當前事件的問題時很有用",
)

print('search.name', search.name)
print('search.description', search.description)
print('search.args', search.args)

search.name Search
search.description 當你需要回答關於當前事件的問題時很有用
search.args {'query': {'title': 'Query', 'type': 'string'}}


## 例外處理


當工具在執行過程中遇到錯誤且未捕捉例外時，程序將崩潰並停止執行。

In [71]:
from langchain_core.tools import ToolException
from langchain.tools import StructuredTool

def search_tool1(query: str) -> str:
    # 模擬正常的查詢邏輯
    if query == "error":
        raise ToolException("搜索工具1不可用。")
    return "搜尋結果"

# 定義一個沒有 handle_tool_error 的工具
search = StructuredTool.from_function(
    func=search_tool1,
    name="Search_tool1",
    description="一個查詢工具"
)

# 嘗試運行工具，將引發未處理的異常
search.run("error")

# 這行程式碼永遠不會被執行，因為程序在前面已經崩潰
print("程序仍然在運行。")


ToolException: 搜索工具1不可用。

透過手動進行例外捕捉

In [ ]:
from langchain_core.tools import ToolException
from langchain.tools import StructuredTool

def search_tool1(query: str) -> str:
    # 模擬正常的查詢邏輯
    if query == "error":
        raise ToolException("搜索工具1不可用。")
    return "搜尋結果"

# 定義一個沒有 handle_tool_error 的工具
search = StructuredTool.from_function(
    func=search_tool1,
    name="Search_tool1",
    description="一個查詢工具"
)

# 手動進行例外捕捉
try:
    search.run("error")
except ToolException as e:
    print(e)

# 進行了例外捕捉，所以程序不會崩潰
print("程序仍然在運行。")


搜索工具1不可用。
程序仍然在運行。


設置 handle_tool_error 為 True，自動進行例外捕捉

In [ ]:
from langchain_core.tools import ToolException
from langchain.tools import StructuredTool

def search_tool1(query: str) -> str:
    # 模擬正常的查詢邏輯
    if query == "error":
        raise ToolException("搜索工具1不可用。")
    return "搜尋結果"

search_with_error_handling = StructuredTool.from_function(
    func=search_tool1,
    name="Search_tool1",
    description="一個查詢工具",
    # 設置為 True
    handle_tool_error=True,
)

# 測試正常查詢情況
result = search_with_error_handling.run("test")
print(result)  # Output: '搜尋結果'

# 測試引發錯誤的情況
result = search_with_error_handling.run("error")
# 預設的例外捕捉
print(result)  # Output: '搜索工具1不可用。'
# 進行了例外捕捉，所以程序不會崩潰
print("進行了例外捕捉，所以程序不會崩潰。")


搜尋結果
搜索工具1不可用。
程序仍然在運行。


設置 handle_tool_error 為字串

In [ ]:
from langchain_core.tools import ToolException
from langchain.tools import StructuredTool

def search_tool1(query: str) -> str:
    # 模擬正常的查詢邏輯
    if query == "error":
        raise ToolException("搜索工具1不可用。")
    return "搜尋結果"

search_with_string_error_handling = StructuredTool.from_function(
    func=search_tool1,
    name="Search_tool1",
    description="一個壞工具",
    handle_tool_error="工具執行過程中發生錯誤，請嘗試使用其他工具。",
)

result = search_with_string_error_handling.run("error")
print(result)

工具執行過程中發生錯誤，請嘗試使用其他工具。


設置 handle_tool_error 為字串時，代理捕捉到異常後返回該字串作為錯誤訊息，而設置為 True 時則會返回異常本身的訊息。

In [ ]:
from langchain_core.tools import ToolException
from langchain.tools import StructuredTool

def search_tool1(query: str) -> str:
    # 模擬正常的查詢邏輯
    if query == "error":
        raise ToolException("搜索工具1不可用。")
    return "搜尋結果"

search_with_string_error_handling = StructuredTool.from_function(
    func=search_tool1,
    name="Search_tool1",
    description="一個查詢工具",
    # 錯誤發生將返回字串
    # handle_tool_error="工具執行過程中發生錯誤，請嘗試使用其他工具。",
    # 錯誤發生將返回錯誤本身
    handle_tool_error=True,
)

# 測試正常查詢情況
result = search_with_string_error_handling.run("test")
print(result)

# 測試引發錯誤的情況
result = search_with_string_error_handling.run("error")
print(result)


搜尋結果
搜索工具1不可用。


異步處理

In [ ]:
import asyncio
from langchain_core.tools import ToolException
from langchain.tools import BaseTool
from typing import Optional, Type
from pydantic import BaseModel, Field
import nest_asyncio

# 避免 Jupyter Notebook 中的事件循環錯誤
nest_asyncio.apply()

class SearchInput(BaseModel):
    query: str = Field(description="要查詢的字串")

class AsyncSearchTool(BaseTool):
    name = "async_search"
    description = "異步查詢工具"
    args_schema: Type[BaseModel] = SearchInput

    def _run(
        self, query: str, run_manager: Optional = None
    ) -> str:
        # 同步版本的查詢邏輯
        if query == "error":
            raise ToolException("同步查詢工具不可用。")
        return "同步查詢結果"

    async def _arun(
        self, query: str, run_manager: Optional = None
    ) -> str:
        # 異步版本的查詢邏輯
        await asyncio.sleep(1)  # 模擬異步操作
        if query == "error":
            raise ToolException("異步查詢工具不可用。")
        return "異步查詢結果"

# 定義一個異步工具
async_search_tool = AsyncSearchTool()

# 測試異步查詢工具
async def test_async_tool():
    try:
        result = await async_search_tool._arun("test")
        print(result)  # Output: '異步查詢結果'
    except ToolException as e:
        print(f"捕捉到異常: {e}")

    try:
        result = await async_search_tool._arun("error")
        print(result)
    except ToolException as e:
        print(f"捕捉到異常: {e}")  # Output: '捕捉到異常: 異步查詢工具不可用。'

# 執行測試異步工具的函數
asyncio.get_event_loop().run_until_complete(test_async_tool())


異步查詢結果
捕捉到異常: 異步查詢工具不可用。
